In [3]:
import pandas as pd
matches = pd.read_csv("matches.csv", index_col=0)
matches.head()


,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,Tottenham,...,Match Report,NaN,18.0,4.0,16.9,1.0,0.0,0.0,2022,Manchester City
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,Norwich City,...,Match Report,NaN,16.0,4.0,17.3,1.0,0.0,0.0,2022,Manchester City
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,Arsenal,...,Match Report,NaN,25.0,10.0,14.3,0.0,0.0,0.0,2022,Manchester City
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,Leicester City,...,Match Report,NaN,25.0,8.0,14.0,0.0,0.0,0.0,2022,Manchester City
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,Southampton,...,Match Report,NaN,16.0,1.0,15.7,1.0,0.0,0.0,2022,Manchester City


In [9]:
matches['round'].value_counts()

round
Matchweek 1     39
Matchweek 16    39
Matchweek 34    39
Matchweek 32    39
Matchweek 31    39
Matchweek 29    39
Matchweek 28    39
Matchweek 26    39
Matchweek 25    39
Matchweek 24    39
Matchweek 23    39
Matchweek 2     39
Matchweek 19    39
Matchweek 17    39
Matchweek 20    39
Matchweek 15    39
Matchweek 5     39
Matchweek 3     39
Matchweek 13    39
Matchweek 12    39
Matchweek 4     39
Matchweek 11    39
Matchweek 10    39
Matchweek 9     39
Matchweek 8     39
Matchweek 14    39
Matchweek 7     39
Matchweek 6     39
Matchweek 30    37
Matchweek 27    37
Matchweek 22    37
Matchweek 21    37
Matchweek 18    37
Matchweek 33    32
Matchweek 35    20
Matchweek 36    20
Matchweek 37    20
Matchweek 38    20
Name: count, dtype: int64

In [10]:
matches.dtypes

date             object
time             object
comp             object
round            object
day              object
venue            object
result           object
gf              float64
ga              float64
opponent         object
xg              float64
xga             float64
poss            float64
attendance      float64
captain          object
formation        object
referee          object
match report     object
notes           float64
sh              float64
sot             float64
dist            float64
fk              float64
pk              float64
pkatt           float64
season            int64
team             object
dtype: object

In [58]:
from sklearn.ensemble import RandomForestClassifier
matches["date"] = pd.to_datetime(matches['date'])
matches['venue_code']=matches['venue'].astype('category').cat.codes
matches['opp_code']=matches['opponent'].astype('category').cat.codes
matches['hour']=matches['time'].str.replace(":.+","",regex=True).astype('int')
matches['day_code'] = matches['date'].dt.dayofweek
matches['target']=(matches['result']=="W").astype("int")
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10,random_state=1) # the higher the estimators the more potentially accurate it iwll b
train= matches[matches["date"]<'2022-01-01']
test= matches[matches['date']<'2021-01-01']
predictors=['venue_code','opp_code','hour', 'day_code']
rf.fit(train[predictors],train['target'])
RandomForestClassifier(n_estimators=50, min_samples_split=10,random_state=1)
preds=rf.predict(test[predictors])
from sklearn.metrics import accuracy_score
acc=accuracy_score(test['target'],preds)
combined = pd.DataFrame(dict(actual=test['target'],prediction=preds))
pd.crosstab(index=combined['actual'], columns=combined['prediction'])
from sklearn.metrics import precision_score
precision_score(test['target'],preds)
grouped_matches= matches.groupby('team')
group= grouped_matches.get_group('Manchester City')
group
def rolling_averages(group, cols, new_cols):
    group = group.sort_values('date')
    rolling_stats= group[cols].rolling(3,closed='left').mean()
    group[new_cols]=rolling_stats
    group=group.dropna(subset=new_cols)
    return group
cols=['gf', 'ga' ,'sh', 'sot', 'dist', 'fk', 'pk', 'pkatt']
new_cols=[f"{c}_rolling" for  c in cols]
rolling_averages(group,cols,new_cols)
matches_rolling=matches.groupby('team').apply(lambda x: rolling_averages(x,cols,new_cols))
matches_rolling=matches_rolling.droplevel('team')
matches_rolling.index= range(matches_rolling.shape[0])
matches_rolling
def make_predictions(data,predictors):
    train= data[data['date']<'2022-01-01']
    test= data[data['date']<'2021-01-01']
    rf.fit(train[predictors],train['target'])
    preds=rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test['target'],prediction=preds), index=test.index)
    precision=precision_score(test['target'],preds)
    return combined, precision
combined, precision= make_predictions(matches_rolling, predictors +new_cols)
precision
combined= combined.merge(matches_rolling[["date","team","opponent","result"]], left_index=True, right_index=True)
combined
class MissingDict(dict):
    __missing__=lambda self,key:key
map_values={"Brighton and Hove Albion":"Brighton",
            "Manchester United": "Manchester Utd",
            "Newcastle United": "Newcastle Utd",
            "Tottenham Hotspur": "Tottenham",
            "West Ham United" : "West Ham",
            "Wolverhampton Wanderers": "Wolves"
}
mapping=MissingDict(**map_values)
combined['new_team']=combined['team'].map(mapping)
merged = combined.merge(combined,left_on=["date","new_team"],right_on=["date","opponent"])
merged
merged[(merged["prediction_x"]==1) & (merged["prediction_y"]==0)]["actual_x"].value_counts()
merged

    

,actual_x,prediction_x,date,team_x,opponent_x,result_x,new_team_x,actual_y,prediction_y,team_y,opponent_y,result_y,new_team_y
0,1,1,2020-10-04,Arsenal,Sheffield Utd,W,Arsenal,0,0,Sheffield United,Arsenal,L,Sheffield United
1,0,0,2020-10-17,Arsenal,Manchester City,L,Arsenal,1,0,Manchester City,Arsenal,W,Manchester City
2,0,0,2020-10-25,Arsenal,Leicester City,L,Arsenal,1,1,Leicester City,Arsenal,W,Leicester City
3,1,0,2020-11-01,Arsenal,Manchester Utd,W,Arsenal,0,0,Manchester United,Arsenal,L,Manchester Utd
4,0,0,2020-11-08,Arsenal,Aston Villa,L,Arsenal,1,1,Aston Villa,Arsenal,W,Aston Villa
...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,0,0,2020-12-12,Wolverhampton Wanderers,Aston Villa,L,Wolves,1,1,Aston Villa,Wolves,W,Aston Villa
210,1,0,2020-12-15,Wolverhampton Wanderers,Chelsea,W,Wolves,0,0,Chelsea,Wolves,L,Chelsea
211,0,0,2020-12-21,Wolverhampton Wanderers,Burnley,L,Wolves,1,1,Burnley,Wolves,W,Burnley
212,0,0,2020-12-27,Wolverhampton Wanderers,Tottenham,D,Wolves,0,0,Tottenham Hotspur,Wolves,D,Tottenham
